# Parameters

In [20]:
image_name = "image1024.jpg"
input_name = "test.txt"
output_name = "results_pc.txt"
fpga_output_name = "results_fpga.txt"
fpga_output_image = "fpga_image.jpg"

# Image Reading

In [15]:
import cv2
import numpy as np
import os

img = cv2.imread(image_name)
N = np.shape(img)[0]
(B,G,R) = cv2.split(img)


# Bayer Image Creation

In [16]:
bayer = np.empty((N, N), np.uint8)

bayer[0::2,0::2] = G[0::2,0::2]
bayer[0::2,1::2] = B[0::2,1::2]
bayer[1::2,0::2] = R[1::2,0::2]
bayer[1::2,1::2] = G[1::2,1::2]

# Bayer test set creation

In [17]:
if(os.path.exists("./sim_1/new/"+input_name)):
    os.remove("./sim_1/new/"+input_name)
with open("./sim_1/new/"+input_name,"x") as f:
    for i in bayer:
        for j in i:
            f.write("{}\n".format(j))

# Debayering Function

In [18]:
def demosaic(inimg,N=16):
    img = np.array([[0 for i in range(N+2) ] for j in range(N+2)])
    img[1:N+1,1:N+1] = inimg
    print(img)
    RGB = (0,0,0)
    res = []
    for i in range(1,N+1):
        for j in range(1,N+1):
            if i%2 and j%2:
                RGB = ((img[i-1][j]+img[i+1][j]) // 2,img[i][j],(img[i][j-1]+img[i][j+1]) //2)
            elif  i%2 and not j%2:
                RGB = ((img[i-1][j-1]+img[i-1][j+1]+img[i+1][j-1]+img[i+1][j+1])//4,(img[i-1][j]+img[i+1][j]+img[i][j-1]+img[i][j+1]) // 4,img[i][j])
            elif not i%2 and j%2:
                RGB = (img[i][j],(img[i-1][j]+img[i+1][j]+img[i][j-1]+img[i][j+1]) // 4,(img[i-1][j-1]+img[i-1][j+1]+img[i+1][j-1]+img[i+1][j+1])//4)
            elif not i%2 and not j%2:
                RGB = ((img[i][j-1]+img[i][j+1]) //2,img[i][j],(img[i-1][j]+img[i+1][j]) // 2)
            res.append(RGB)
    return res

# Bayer from a txt file

In [ ]:
with open("./sim_1/new/"+input_name,"r") as f:
    temp = f.readlines()
    bayer = []
    cnt = 0
    for i in range(N):
        pup = []
        for j in range(N):
            t = temp[cnt]
            pup.append(t)
            cnt +=1
        bayer.append(pup)

# Debayering results from PC

In [19]:
if(os.path.exists(output_name)):
    os.remove(output_name)
with open(output_name,"x") as f:
    for i in demosaic(bayer,N):
        f.write("{} {} {}\n".format(i[0],i[1],i[2]))


[[  0   0   0 ...   0   0   0]
 [  0  45  78 ...  58  70   0]
 [  0  82  56 ...  75  56   0]
 ...
 [  0  43  52 ...  75 167   0]
 [  0  10  45 ...  46  75   0]
 [  0   0   0 ...   0   0   0]]


# Image reconstruction from FPGA results

In [13]:
with open("./sim_1/new/"+fpga_output_name,"r") as f:
    temp = f.readlines()
    debayer = []
    cnt = 0
    for i in range(N):
        pup = []
        for j in range(N):
            t = temp[cnt].split(" ")
            pup.append([t[2],t[1],t[0]])
            cnt +=1
        debayer.append(pup)

cv2.imwrite(fpga_output_image,np.array(debayer,dtype="uint8"))

True